In [2]:
from nwbinspector import inspect_nwb

results = list(inspect_nwb(nwbfile_path="basics_tutorial.nwb"))

In [5]:
from pynwb import NWBFile
import pandas as pd
from pynwb import register_class, load_namespaces
from pynwb.core import NWBContainer, NWBDataInterface
from pynwb.base import TimeSeries
from collections.abc import Iterable
from pynwb.device import  Device
from hdmf.utils import docval, popargs, get_docval, popargs_to_dict
from pynwb.core import DynamicTable, VectorData
import numpy as np
from hdmf.utils import docval, getargs, popargs, popargs_to_dict, get_docval, call_docval_func
import warnings
from pynwb.io.core import NWBContainerMapper
from pynwb import register_map
from pynwb.base import TimeSeries, TimeSeriesReferenceVectorData, TimeSeriesReference
from pynwb.file import MultiContainerInterface, NWBContainer
from hdmf.build import ObjectMapper
from hdmf.common.io.table import DynamicTableMap
import matplotlib.pyplot as plt
from pynwb import NWBFile

# @docval(get_docval(NWBFile.__init__))
def new_init(self, **kwargs):
    NWBFile.__init__(self, **kwargs)

NWBFile.__init__ = new_init
#

In [5]:
from pynwb.spec import NWBDatasetSpec, NWBNamespaceBuilder, NWBGroupSpec
from pynwb.spec import NWBNamespaceBuilder, export_spec, NWBGroupSpec, NWBAttributeSpec
from pynwb.spec import NWBDatasetSpec
from pynwb.spec import NWBNamespaceBuilder, NWBGroupSpec, NWBAttributeSpec, NWBLinkSpec, NWBDtypeSpec, NWBDatasetSpec, NWBRefSpec
from datetime import datetime
from dateutil.tz import tzlocal

from pynwb import NWBFile

ns_path = "mylab.namespace.yaml"
ext_source = "mylab.extensions.yaml"

ns_builder = NWBNamespaceBuilder('Extension for use in my Lab', "mylab", version='0.1.0')
# Now we define the data structures. We use `NWBDataInterface` as the base type,
# which is the most primitive type you are likely to use as a base. The name of the
# class is `CorticalSurface`, and it requires two matrices, `vertices` and
# `faces`.
ns_builder.include_type('NWBContainer', namespace='core')
ns_builder.include_type('Device', namespace='core')
ns_builder.include_type('NWBDataInterface', namespace='core')
ns_builder.include_type('TimeSeries', namespace='core')
ns_builder.include_type('DynamicTable', namespace='core')
ns_builder.include_type('VectorData', namespace='core')
ns_builder.include_type('VectorIndex', namespace='core')
ns_builder.include_type("DynamicTableRegion", namespace="core")
ns_builder.include_type("TimeSeriesReferenceVectorData", namespace="core")
#
series = NWBGroupSpec(
    name='series',
    neurodata_type_def='Series',
    neurodata_type_inc='TimeSeries',
    doc=('PhotostimulationSeries container'),
)
ns_builder.add_spec(ext_source, series)

# series_ref=NWBDatasetSpec(name='series',
#                          quantity='?',
#                            doc='...',
#                          neurodata_type_inc='VectorData',
#                          dtype=[
#                              NWBDtypeSpec('series', 'column for foo', dtype=NWBRefSpec(target_type='Series', reftype='object'))
#                          ])

# series_ref = NWBDatasetSpec(name='time_series', doc='...', neurodata_type_inc='TimeSeries')#, neurodata_type_inc='TimeSeriesReferenceVectorData', dtype=NWBRefSpec(target_type='TimeSeries', reftype='object'), doc=('...'), quantity='?')

#, neurodata_type_inc='TimeSeriesReferenceVectorData',

events = NWBGroupSpec(
    neurodata_type_def='Events',
    neurodata_type_inc='DynamicTable',
    doc='A list of timestamps, stored in seconds, of an event.',
    # datasets=[NWBDatasetSpec(name='series', doc='pres', quantity='?', neurodata_type_inc='TimeSeriesReferenceVectorData')]
    # datasets=[NWBDatasetSpec(name='series', doc='pres', quantity='?',  neurodata_type_inc='VectorData', dtype=NWBRefSpec(target_type='Series', reftype='object'))]
# datasets=[NWBDatasetSpec(name='time_series', doc='...', neurodata_type_inc='TimeSeriesReferenceVectorData')]
    # groups=[NWBGroupSpec(name='time_series', doc='...', datasets=[series_ref])]

    groups=[series]
)

ns_builder.add_spec(ext_source, events)
ns_builder.export(ns_path)

In [6]:
from pynwb import get_class, load_namespaces
from pynwb import register_class, load_namespaces
from pynwb.core import NWBContainer, NWBDataInterface
from pynwb.base import TimeSeries
from collections.abc import Iterable
from pynwb.device import  Device
from hdmf.utils import docval, popargs, get_docval, popargs_to_dict
from pynwb.core import DynamicTable, VectorData
import numpy as np
from hdmf.utils import docval, getargs, popargs, popargs_to_dict, get_docval, call_docval_func
import warnings
from pynwb.base import TimeSeriesReferenceVectorData

from pynwb.io.core import NWBContainerMapper
from pynwb import register_map

from hdmf.build import LinkBuilder


ns_path = "mylab.namespace.yaml"
load_namespaces(ns_path)


@register_class('Series', 'mylab')
class Series(TimeSeries):

    @docval(*get_docval(TimeSeries.__init__)
    )
    def __init__(self, **kwargs):
       super().__init__(**kwargs)

@register_class('Events', 'mylab')
class Events(DynamicTable):
    """
    A list of timestamps, stored in seconds, of an event.
    """

    __columns__ = (
            {'name': 'time_series', 'description': 'PatchClampSeries with the same sweep number' },
    )

    @docval({'name': 'name', 'type': str, 'doc': 'name of this SweepTable', 'default': 'sweep_table'},
            {'name': 'description', 'type': str, 'doc': 'Description of this SweepTable',
             'default': "A sweep table groups different PatchClampSeries together."},
            *get_docval(DynamicTable.__init__, 'id', 'columns', 'colnames'))
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    @docval({'name': 'time_series', 'type': Series, 'doc': 'name of this TimeIntervals', 'default': None},
    allow_extra=True)
    def add_event_type(self, **kwargs):
        """Add an event type as a row to this table."""
        # data = {'time_series': kwargs['time_series']}
        super().add_row(**kwargs)


In [7]:
series = Series(name='test', description='test d', data = [0, 1, 2, 3, 4, 5], unit='sec', rate=10.)

sp = Events()#, stimulus_method='method', sweeping_method='sweeping')
sp.add_event_type(time_series=Series(name='test', description='test d', data = [0, 1, 2, 3, 4, 5], unit='sec', rate=10.))
sp.add_event_type(time_series=Series(name='test', description='test d', data = [0, 1, 2, 3, 4, 5], unit='sec', rate=10.))
# df = sp.to_dataframe()
# print(df)
from pynwb import NWBHDF5IO

nwbfile = NWBFile(
    'my first synthetic recording',
    'EXAMPLE_ID',
    datetime.now(tzlocal()),
)

nwbfile.add_acquisition(sp)
with NWBHDF5IO("basics_tutorial.nwb", "w") as io:
    # io.write(series)
    io.write(nwbfile)

with NWBHDF5IO("basics_tutorial.nwb", "r", load_namespaces=True) as io:
    read_nwbfile = io.read()
    a = read_nwbfile.acquisition['test_presentation_table']
    print(a.time_series)


ContainerConfigurationError: Events 'sweep_table' does not have attribute 'series' for mapping to spec: {'dtype': {'target_type': 'Series', 'reftype': 'object'}, 'doc': 'pres', 'name': 'series', 'quantity': '?', 'neurodata_type_inc': 'VectorData'}

In [62]:
a.columns

()

In [8]:
col1 = VectorData(
    name='col1',
    description='column #1',
    data=[1, 2],
)

col2 = VectorData(
    name='col2',
    description='column #2',
    data=[series, series],
)

# this table will have two rows with ids 0 and 1
table = DynamicTable(
    name='my table',
    description='an example table',
    columns=[col1, col2],
)

df = table.to_dataframe()
print(df)


with NWBHDF5IO("basics_tutorial.nwb", "w") as io:
    # io.write(series)
    io.write(table)

with NWBHDF5IO("basics_tutorial.nwb", "r", load_namespaces=True) as io:
    read_nwbfile = io.read()
    # a = read_nwbfile.acquisition['test_presentation_table']
    # print(a.time_series)
    print('')

    col1                                               col2
id                                                         
0      1  test __main__.Series at 0x140636842405520\nFie...
1      2  test __main__.Series at 0x140636842405520\nFie...


ReferenceTargetNotBuiltError: col2 (my table/col2): Could not find already-built Builder for Series 'test' in BuildManager

In [ ]:

from datetime import datetime
from dateutil.tz import tzlocal
from pynwb import load_namespaces


nwbfile.add_acquisition(events)

with NWBHDF5IO("basics_tutorial.nwb", "w") as io:
    io.write(nwbfile)

with NWBHDF5IO("basics_tutorial.nwb", "r", load_namespaces=True) as io:
    read_nwbfile = io.read()


print('')

In [20]:
io.close()

In [ ]:
with NWBHDF5IO("basics_tutorial.nwb", "r", load_namespaces=True) as io:
    read_nwbfile = io.read()

In [10]:
import file_classes

from datetime import datetime
from dateutil.tz import tzlocal
from pynwb import load_namespaces

from pynwb import NWBFile

name = "test"
ns_path = name + ".namespace.yaml"
load_namespaces(ns_path)

from pynwb import register_class, load_namespaces
from pynwb.core import NWBContainer, NWBDataInterface
from pynwb.base import TimeSeries
from collections.abc import Iterable
from pynwb.device import  Device
from hdmf.utils import docval, popargs, get_docval, popargs_to_dict
from pynwb.core import DynamicTable, VectorData
import numpy as np
from hdmf.utils import docval, getargs, popargs, popargs_to_dict, get_docval, call_docval_func
import warnings
from pynwb.io.core import NWBContainerMapper
from pynwb import register_map
from pynwb.base import TimeSeries, TimeSeriesReferenceVectorData, TimeSeriesReference

from file_classes import StimulusPresentation



In [11]:
help(StimulusPresentation)

Help on class StimulusPresentation in module file_classes:

class StimulusPresentation(hdmf.common.table.DynamicTable)
 |  StimulusPresentation(*args, **kwargs)
 |  
 |  Table for storing Epoch data
 |  
 |  Method resolution order:
 |      StimulusPresentation
 |      hdmf.common.table.DynamicTable
 |      hdmf.container.Container
 |      hdmf.container.AbstractContainer
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(*args, **kwargs)
 |      __init__(name, description, id=None, columns=None, colnames=None)
 |      
 |      
 |      
 |      Args:
 |          name (:py:class:`~str`): name of this TimeIntervals
 |          description (:py:class:`~str`): Description of this TimeIntervals
 |          id (:py:class:`~numpy.ndarray` or :py:class:`~list` or :py:class:`~tuple` or :py:class:`~Dataset` or :py:class:`~zarr.core.Array` or :py:class:`~hdmf.utils.StrDataset` or :py:class:`~hdmf.query.HDMFDataset` or :py:class:`~hdmf.data_utils.AbstractDataChunkIterator` o

In [9]:
print(StimulusPresentation.__dict__)

{'__module__': '__main__', '__init__': <function StimulusPresentation.__init__ at 0x7fb5391c5750>, '__doc__': None, '__abstractmethods__': frozenset(), '_abc_impl': <_abc._abc_data object at 0x7fb528f62e80>, 'constructor_args': {'name': <function ObjectMapper.get_container_name at 0x7fb528cc81f0>}, 'obj_attrs': {}}


In [13]:
print(StimulusPresentation.type_hierarchy())

(<class 'file_classes.StimulusPresentation'>, <class 'hdmf.common.table.DynamicTable'>, <class 'hdmf.container.Container'>, <class 'hdmf.container.AbstractContainer'>, <class 'object'>)


In [11]:
from hdmf.common import DynamicTable

users_table = DynamicTable(
    name='users',
    description='a table containing data/metadata about users, one user per row',
)

users_table.add_column(
    name='first_name',
    description='the first name of the user',
)

users_table.add_column(
    name='last_name',
    description='the last name of the user',
)

users_table.add_column(
    name='phone_number',
    description='the phone number of the user',
    index=False,
)

ts = TimeSeries(name='test', description='test d', data = [0, 1, 2, 3, 4, 5], unit='sec', rate=10.)

# id will be set to 0 automatically
users_table.add_row(
    first_name='Grace',
    last_name='Hopper',
    phone_number=ts,
)

# id will be set to 1 automatically
users_table.add_row(
    first_name='Alan',
    last_name='Turing',
    phone_number=ts,
)

with NWBHDF5IO("basics_tutorial.nwb", "w") as io:
    # io.write(series)
    io.write(ts)
    io.write(users_table)

with NWBHDF5IO("basics_tutorial.nwb", "r", load_namespaces=True) as io:
    read_nwbfile = io.read()
    # a = read_nwbfile.acquisition['test_presentation_table']
    # print(a.time_series)
    print('')


In [1]:
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget

io = NWBHDF5IO('basics_tutorial.nwb', mode='r', load_namespaces=True)
nwb = io.read()

nwb2widget(nwb)

print('')

In [13]:
io.close()